## Building A Chatbot
In this notebook We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This notebook will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FA2207C690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FA2207ED50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Namaste , My name is Anish and I am a Computer Science Student.")])

AIMessage(content='Namaste Anish! It\'s nice to meet you. \n\nAs a fellow "student" of sorts (though I learn in a very different way!), I\'m always happy to chat with someone interested in computer science. What are you studying these days?  Anything particularly interesting you\'re working on?  😊  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 24, 'total_tokens': 94, 'completion_time': 0.127272727, 'prompt_time': 0.002110652, 'queue_time': 0.017288486999999998, 'total_time': 0.129383379}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2917ecc5-88c7-4ec5-bbdf-569e9f281823-0', usage_metadata={'input_tokens': 24, 'output_tokens': 70, 'total_tokens': 94})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Namaste , My name is Anish and I am a Computer Science Student."),
        AIMessage(content="Namaste Anish! It\'s nice to meet you. \n\nAs a fellow student of sorts (though I learn in a very different way!), I\'m always happy to chat with someone interested in computer science. What are you studying these days?  Anything particularly interesting you\'re working on?  😊  \n\n"),
        HumanMessage(content="Hello there, What's my name and what do I do?")
    ]
)

AIMessage(content="You are Anish, a Computer Science Student! \n\nIs there anything else you'd like to tell me about yourself, or would you like to ask me something else? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 113, 'total_tokens': 154, 'completion_time': 0.074545455, 'prompt_time': 0.005720745, 'queue_time': 0.020837075, 'total_time': 0.0802662}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9098e655-5a2e-423a-a94c-93518ae11bdd-0', usage_metadata={'input_tokens': 113, 'output_tokens': 41, 'total_tokens': 154})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Namaste , My name is Anish and I am a Computer Science Student.")],
    config=config
)

In [9]:
response.content

"Namaste Anish! It's nice to meet you. \n\nAs a computer science student, what areas of computer science are you most interested in?\n\nI'm always eager to learn more about what people are passionate about in the field.\n"

In [11]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response2.content

"Your name is Anish.  \n\nI'm ready for another question whenever you are! 😄 \n"

In [12]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response3=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response3.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Ram.")],
    config=config1
)
response.content

"Hello Ram, it's nice to meet you!  \n\nIs there anything I can help you with today?\n"

In [14]:
response4=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response4.content

'Your name is Ram.  I remember! \n\nDo you have any other questions for me, Ram? 😊  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
chain.invoke({"messages":[HumanMessage(content="Namaste! My name is Anish")]})

AIMessage(content="Namaste Anish!  \n\nIt's lovely to meet you. I'm happy to help with any questions you have.  \n\nJust ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 34, 'total_tokens': 72, 'completion_time': 0.069090909, 'prompt_time': 0.002329887, 'queue_time': 0.017140970000000002, 'total_time': 0.071420796}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c783a9e8-c17f-4a4c-a1af-1d41d60591eb-0', usage_metadata={'input_tokens': 34, 'output_tokens': 38, 'total_tokens': 72})

In [18]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [20]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Namaste! My name is Anish")],
    config=config
)

response.content

"Namaste Anish!  It's a pleasure to meet you. \n\nIs there anything I can help you with today? 😊  \n\n"

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Anish. 😊  \n\nI remember!  It's nice to meet you, Anish. \n\n\n\n"

In [42]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [49]:
response=chain.invoke({"messages":[HumanMessage(content="Namaste! My name is Anish")],"language":"Nepali"})
response.content

'नमस्ते! अनिश, आनन्द छ। मैले कसैले पनि सहायता गर्न सक्छु जसरी म सक्छु। के तपाईंलाई केही प्रश्न छ ? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [43]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [44]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Namaste! My name is Anish")],"language":"English"},
    config=config
)
repsonse.content

'नमस्ते अनिष! 😊\n\nआपको नाम थाहा पाएर मलाई खुशी छ! \n\n के मलाई केही गर्न सक्छु?  \n'

In [50]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Nepali"},
    config=config,
)

In [53]:
response.content

'आपको नाम अनिष हो। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [55]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\anish\OneDrive\Desktop\Langchain Practice\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anish\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [56]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor?  🍦\n"

In [57]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "what is 2 + 2" 🧮  \n\n\n\nLet me know if you\'d like to try another one! 😊\n'

In [58]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [59]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or personal information about you, including your name. 😊\n\nWould you like to tell me your name?\n"

In [60]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations. If you'd like to ask me a math problem, I'd be happy to help! 😊  \n\n"